# Analysing the spread of Turkish Kebab restaurants in Milan, Italy

## Steps:

1. Creating a dataframe of neighborhoods in Milano, from Wikipedia page
2. Getting the geographical coordinates of the neighborhoods
3. Retrieving the venue data for the neighborhoods from Foursquare API
4. Exploring and clustering the neighborhoods
5. Selecting the best cluster to open a Turkish Kebab restaurant

### Importing libraries

In [5]:
import numpy as np 
import pandas as pd 
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
from bs4 import BeautifulSoup 
from sklearn.cluster import KMeans
import json # handle JSON files
from geopy.geocoders import Nominatim # convert to latitude and longitude values
!pip install geocoder
import geocoder 
import requests 
from pandas.io.json import json_normalize 
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium

### Scrap data from Wikipedia into a DataFrame

In [6]:
# Getting information about Milan
data = requests.get("https://en.wikipedia.org/wiki/Category:Districts_of_Milan").text

In [7]:
# creating beautifulsoup object from html
soup = BeautifulSoup(data, 'html.parser')
soup

<!DOCTYPE html>

<html class="client-nojs" dir="ltr" lang="en">
<head>
<meta charset="utf-8"/>
<title>Category:Districts of Milan - Wikipedia</title>
<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"24c92bc4-5bbf-47f4-a0ac-78254885364e","wgCSPNonce":!1,"wgCanonicalNamespace":"Category","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":14,"wgPageName":"Category:Districts_of_Milan","wgTitle":"Districts of Milan","wgCurRevisionId":899922759,"wgRevisionId":899922759,"wgArticleId":19192884,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Commons category link is on Wikidata","Subdivisions of Milan","City districts"],"wgPageContentLanguage":"en","wgPageContentMo

In [8]:
neighborhoodList = []

# append neighborhood data into the list
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhoodList.append(row.text)

In [9]:
# create a new pandas DataFrame from neighborhoods
mi_df = pd.DataFrame({"Neighborhood": neighborhoodList})

mi_df.head()

,Neighborhood
0,Affori
1,Assiano
2,Baggio (district of Milan)
3,Barona
4,Bicocca (district of Milan)


### Getting the geographical coordinates

In [10]:
# Function to obtain the locations of the neighborhood in Milan.
def get_latlng(neighborhood):
    lat_lng_coords = None
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Milan, Italy'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [11]:
# call the previous function to get the coordinates, store in a coords list
coords = [ get_latlng(neighborhood) for neighborhood in mi_df["Neighborhood"].tolist() ]

coords

[[45.51410000000004, 9.173530000000028],
 [45.45059636041839, 9.061648559640952],
 [45.46324000000004, 9.092700000000036],
 [45.433710000000076, 9.15160000000003],
 [45.52149000000003, 9.213260000000048],
 [45.503130000000056, 9.161220000000071],
 [45.515550000000076, 9.150940000000048],
 [45.47472000000005, 9.189900000000023],
 [45.52825000000007, 9.180710000000033],
 [45.456180000000074, 9.224880000000041],
 [45.54799234966403, 8.994257200554808],
 [45.50198373621629, 9.264641249871804],
 [45.41719000000006, 9.23971000000006],
 [45.500860000000046, 9.265130000000056],
 [45.503460000000075, 9.248800000000074],
 [45.47708000000006, 9.226600000000076],
 [45.52631000000008, 9.158870000000036],
 [45.38412135138062, 9.13262883708055],
 [45.51054000000005, 9.24386000000004],
 [45.504110000000026, 9.176470000000052],
 [45.49234000000007, 9.078520000000026],
 [45.45975000000004, 9.24690000000004],
 [45.496710000000064, 9.114840000000072],
 [45.50469000000004, 9.136970000000076],
 [45.49631000

In [12]:
# create temporary dataframe
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

mi_df['Latitude'] = df_coords['Latitude']
mi_df['Longitude'] = df_coords['Longitude']

# check the neighborhoods and the coordinates in main dataframe
print(mi_df.shape)
mi_df

(78, 3)


,Neighborhood,Latitude,Longitude
0,Affori,45.514100,9.173530
1,Assiano,45.450596,9.061649
2,Baggio (district of Milan),45.463240,9.092700
3,Barona,45.433710,9.151600
4,Bicocca (district of Milan),45.521490,9.213260
5,Bovisa,45.503130,9.161220
6,Bovisasca,45.515550,9.150940
7,Brera (district of Milan),45.474720,9.189900
8,Bruzzano,45.528250,9.180710
9,Calvairate,45.456180,9.224880


In [13]:
# save it
mi_df.to_csv("mi_df.csv", index=False)

### Creating a map of Milan with its neighborhoods superimposed on top

In [15]:
# get the coordinates of Milan
address = 'Milan, Italy'

geolocator = Nominatim(user_agent="tbncap")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Milan,Italy {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Milan,Italy 45.4668, 9.1905.


In [16]:
# create a map
map_mi = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers 
for lat, lng, neighborhood in zip(mi_df['Latitude'], mi_df['Longitude'], mi_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_mi)  
    
map_mi

In [17]:
# save it as html file
map_mi.save('map_mi.html')

### Exploring the neighborhoods using the Foursquare API

In [18]:
CLIENT_ID = '0LL1IESTAJ0RR1KMLGQDKW5WDJ0R2FY224NP0VZBB20CC4KP'
CLIENT_SECRET = '2TM1S3QF0RXQWQV3DFCVXRX4X50J1IGBAO0I5NUQXDGAUPGP'
VERSION = '20180602'

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 0LL1IESTAJ0RR1KMLGQDKW5WDJ0R2FY224NP0VZBB20CC4KP
CLIENT_SECRET:2TM1S3QF0RXQWQV3DFCVXRX4X50J1IGBAO0I5NUQXDGAUPGP


Retrieveing the top 200 venues that are within a radius of 2000 meters

In [19]:
radius = 2000
LIMIT = 200

venues = []

for lat, long, neighborhood in zip(mi_df['Latitude'], mi_df['Longitude'], mi_df['Neighborhood']):
    
    # create API request 
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [20]:
# convert venues list to a new DataFrame (venues_df)
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(6036, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Affori,45.5141,9.17353,Esselunga,45.512380,9.173461,Supermarket
1,Affori,45.5141,9.17353,Al Paradiso Della Pizza,45.511351,9.175416,Pizza Place
2,Affori,45.5141,9.17353,Parco di Villa Litta,45.516414,9.167165,Park
3,Affori,45.5141,9.17353,Osteria del biliardo,45.514741,9.169345,Pool Hall
4,Affori,45.5141,9.17353,Istanbul Kebab,45.510421,9.176128,Kebab Restaurant


Checking how many venues are returned for each neighborhood

In [21]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
Affori,55,55,55,55,55,55
Assiano,8,8,8,8,8,8
Baggio (district of Milan),40,40,40,40,40,40
Barona,18,18,18,18,18,18
Bicocca (district of Milan),100,100,100,100,100,100
Bovisa,100,100,100,100,100,100
Bovisasca,95,95,95,95,95,95
Brera (district of Milan),100,100,100,100,100,100
Bruzzano,75,75,75,75,75,75


Let's see how many unique categories are there

In [22]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 277 uniques categories.


In [23]:
# print out the list of categories
venues_df['VenueCategory'].unique()[:50]

array(['Supermarket', 'Pizza Place', 'Park', 'Pool Hall',
       'Kebab Restaurant', 'Soccer Field', 'Hotel Bar', 'Brewery',
       'Ballroom', 'Health Food Store', 'Hobby Shop', 'Hotel',
       'Gym / Fitness Center', 'Italian Restaurant',
       'Paper / Office Supplies Store', 'Theater', 'Gym',
       'Ice Cream Shop', 'Seafood Restaurant', 'Piadineria',
       'Japanese Restaurant', 'Bistro', 'Creperie', 'Art Gallery', 'Café',
       'Steakhouse', 'Food', 'Pub', 'Bar', 'Sushi Restaurant',
       'Sicilian Restaurant', 'Bookstore', 'Indian Restaurant',
       'Chinese Restaurant', 'Diner', 'Plaza', 'Arts & Crafts Store',
       'Electronics Store', "Men's Store", 'Shoe Store', 'Clothing Store',
       'Mediterranean Restaurant', 'Flower Shop', 'Neighborhood',
       'Food Court', 'Playground', 'Water Park', 'Pool', 'Cheese Shop',
       'Campanian Restaurant'], dtype=object)

In [24]:
# check if the results contain "Kebab Restaurant"
"Neighborhood" in venues_df['VenueCategory'].unique()

True

## Analyzing each neighborhood

In [25]:
# one hot encoding
mi_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
mi_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [mi_onehot.columns[-1]] + list(mi_onehot.columns[:-1])
mi_onehot = mi_onehot[fixed_columns]

print(mi_onehot.shape)
mi_onehot.head()

(6036, 278)


,Neighborhoods,Abruzzo Restaurant,Accessories Store,Adult Education Center,African Restaurant,Agriturismo,Airport,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Amphitheater,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Garage,Bakery,Ballroom,Bar,Basketball Court,Basketball Stadium,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Buffet,Burger Joint,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Calabria Restaurant,Camera Store,Campanian Restaurant,Canal,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Casino,Castle,Cave,Cemetery,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,City,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,Comedy Club,Comic Shop,Convenience Store,Convention Center,Cosmetics Shop,Creperie,Cultural Center,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doctor's Office,Dog Run,Doner Restaurant,Eastern European Restaurant,Electronics Store,Emilia Restaurant,Event Space,Farm,Farmers Market,Fast Food Restaurant,Film Studio,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Football Stadium,Fountain,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Health Food Store,Historic Site,Hobby Shop,Hockey Arena,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kebab Restaurant,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Lebanese Restaurant,Lombard Restaurant,Lounge,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Moroccan Restaurant,Motel,Motorcycle Shop,Mountain,Movie Theater,Multiplex,Museum,Music School,Music Store,Music Venue,Neighborhood,Nightclub,Noodle House,Office,Opera House,Optical Shop,Other Nightlife,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Photography Lab,Piadineria,Pizza Place,Planetarium,Platform,Playground,Plaza,Pool,Pool Hall,Pub,Puglia Restaurant,Racecourse,Radio Station,Ramen Restaurant,Record Shop,Recording Studio,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Rest Area,Restaurant,River,Road,Rock Club,Roman Restaurant,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sardinian Restaurant,Sauna / Steam Room,Scenic Lookout,Science Museum,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Shopping Plaza,Sicilian Restaurant,Skate Park,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Sri Lankan Restaurant,Stables,Stadium,Steakhouse,Street Art,Strip Club,Supermarket,Sushi Restaurant,Szechuan Restaurant,Tapas Restaurant,Tea Room,Tennis Court,Tennis Stadium,Thai Restaurant,Theater,Theme Park Ride / Attraction,Thrift / Vintage Store,Toll Plaza,Toy / Game Store,Track,Train Station,Tram Station,Trattoria/Osteria,Tuscan Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Volleyball Court,Warehouse Store,Water Park,Wine Bar,Wine Shop,Winery,Women's Store
0,Affori,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

Now let's group the frequencies by neighborhood and category

In [26]:
mi_grouped = mi_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(mi_grouped.shape)
mi_grouped

(78, 278)


,Neighborhoods,Abruzzo Restaurant,Accessories Store,Adult Education Center,African Restaurant,Agriturismo,Airport,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Amphitheater,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Garage,Bakery,Ballroom,Bar,Basketball Court,Basketball Stadium,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Buffet,Burger Joint,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Calabria Restaurant,Camera Store,Campanian Restaurant,Canal,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Casino,Castle,Cave,Cemetery,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,City,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,Comedy Club,Comic Shop,Convenience Store,Convention Center,Cosmetics Shop,Creperie,Cultural Center,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doctor's Office,Dog Run,Doner Restaurant,Eastern European Restaurant,Electronics Store,Emilia Restaurant,Event Space,Farm,Farmers Market,Fast Food Restaurant,Film Studio,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Football Stadium,Fountain,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Health Food Store,Historic Site,Hobby Shop,Hockey Arena,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kebab Restaurant,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Lebanese Restaurant,Lombard Restaurant,Lounge,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Moroccan Restaurant,Motel,Motorcycle Shop,Mountain,Movie Theater,Multiplex,Museum,Music School,Music Store,Music Venue,Neighborhood,Nightclub,Noodle House,Office,Opera House,Optical Shop,Other Nightlife,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Photography Lab,Piadineria,Pizza Place,Planetarium,Platform,Playground,Plaza,Pool,Pool Hall,Pub,Puglia Restaurant,Racecourse,Radio Station,Ramen Restaurant,Record Shop,Recording Studio,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Rest Area,Restaurant,River,Road,Rock Club,Roman Restaurant,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sardinian Restaurant,Sauna / Steam Room,Scenic Lookout,Science Museum,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Shopping Plaza,Sicilian Restaurant,Skate Park,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Sri Lankan Restaurant,Stables,Stadium,Steakhouse,Street Art,Strip Club,Supermarket,Sushi Restaurant,Szechuan Restaurant,Tapas Restaurant,Tea Room,Tennis Court,Tennis Stadium,Thai Restaurant,Theater,Theme Park Ride / Attraction,Thrift / Vintage Store,Toll Plaza,Toy / Game Store,Track,Train Station,Tram Station,Trattoria/Osteria,Tuscan Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Volleyball Court,Warehouse Store,Water Park,Wine Bar,Wine Shop,Winery,Women's Store
0,Affori,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.00000

In [27]:
# Let's find the ones in  Kebab Restaurant
len(mi_grouped[mi_grouped["Kebab Restaurant"] > 0])

10

In [28]:
# Create a new DataFrame for Kebab Restaurant data only
mi_kebab = mi_grouped[["Neighborhoods","Kebab Restaurant"]]
mi_kebab.head()

,Neighborhoods,Kebab Restaurant
0,Affori,0.018182
1,Assiano,0.000000
2,Baggio (district of Milan),0.000000
3,Barona,0.000000
4,Bicocca (district of Milan),0.010000


### Clustering the neighborhoods

Let's run the k-means to cluster the neighborhoods in Milan into 3 clusters

In [29]:
kclusters = 3

mi_clustering = mi_kebab.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(mi_clustering)

kmeans.labels_[0:10]

array([1, 0, 0, 0, 2, 1, 2, 0, 0, 0])

In [30]:
# create a new data frame that includes the cluster and the 10 locations for each neighborhood.
mi_merged = mi_kebab.copy()

mi_merged["Cluster Labels"] = kmeans.labels_

mi_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
mi_merged.head()

,Neighborhood,Kebab Restaurant,Cluster Labels
0,Affori,0.018182,1
1,Assiano,0.000000,0
2,Baggio (district of Milan),0.000000,0
3,Barona,0.000000,0
4,Bicocca (district of Milan),0.010000,2


In [31]:
mi_merged = mi_merged.join(mi_df.set_index("Neighborhood"), on="Neighborhood")

print(mi_merged.shape)
mi_merged.head()

(78, 5)


,Neighborhood,Kebab Restaurant,Cluster Labels,Latitude,Longitude
0,Affori,0.018182,1,45.514100,9.173530
1,Assiano,0.000000,0,45.450596,9.061649
2,Baggio (district of Milan),0.000000,0,45.463240,9.092700
3,Barona,0.000000,0,45.433710,9.151600
4,Bicocca (district of Milan),0.010000,2,45.521490,9.213260


In [32]:
print(mi_merged.shape)
mi_merged.sort_values(["Cluster Labels"], inplace=True)
mi_merged

(78, 5)


,Neighborhood,Kebab Restaurant,Cluster Labels,Latitude,Longitude
38,Ortica,0.000000,0,45.470960,9.241600
70,Stazione di Milano Centrale,0.000000,0,45.485670,9.204050
37,Nosedo,0.000000,0,45.433810,9.221370
76,Vigentino,0.000000,0,45.433720,9.201040
39,Ponte Lambro (district of Milan),0.000000,0,45.442400,9.264200
40,Porta Garibaldi (Milan),0.000000,0,45.480650,9.187310
41,Porta Genova,0.000000,0,45.457900,9.174570
42,Porta Lodovica,0.000000,0,45.453180,9.189290
43,Porta Magenta,0.000000,0,45.444469,8.886528
44,Porta Monforte,0.000000,0,45.467229,9.202485


Now let's visualize the resulting clusters

In [33]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers
markers_colors = []
for lat, lon, poi, cluster in zip(mi_merged['Latitude'], mi_merged['Longitude'], mi_merged['Neighborhood'], mi_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [34]:
# save the map
map_clusters.save('map_clusters.html')

### Examining the clusters

Cluster 0

In [36]:
mi_merged.loc[mi_merged['Cluster Labels'] == 0]

,Neighborhood,Kebab Restaurant,Cluster Labels,Latitude,Longitude
38,Ortica,0.0,0,45.470960,9.241600
70,Stazione di Milano Centrale,0.0,0,45.485670,9.204050
37,Nosedo,0.0,0,45.433810,9.221370
76,Vigentino,0.0,0,45.433720,9.201040
39,Ponte Lambro (district of Milan),0.0,0,45.442400,9.264200
40,Porta Garibaldi (Milan),0.0,0,45.480650,9.187310
41,Porta Genova,0.0,0,45.457900,9.174570
42,Porta Lodovica,0.0,0,45.453180,9.189290
43,Porta Magenta,0.0,0,45.444469,8.886528
44,Porta Monforte,0.0,0,45.467229,9.202485


Cluster 1

In [37]:
mi_merged.loc[mi_merged['Cluster Labels'] == 1]

,Neighborhood,Kebab Restaurant,Cluster Labels,Latitude,Longitude
0,Affori,0.018182,1,45.51410,9.17353
36,Niguarda,0.020000,1,45.51840,9.19201
5,Bovisa,0.020000,1,45.50313,9.16122


Cluster 2

In [38]:
mi_merged.loc[mi_merged['Cluster Labels'] == 2]

,Neighborhood,Kebab Restaurant,Cluster Labels,Latitude,Longitude
55,Prato Centenaro,0.010000,2,45.50671,9.199210
25,Giambellino-Lorenteggio,0.011236,2,45.44467,9.129891
19,Dergano,0.010000,2,45.50411,9.176470
6,Bovisasca,0.010526,2,45.51555,9.150940
4,Bicocca (district of Milan),0.010000,2,45.52149,9.213260
67,San Cristoforo sul Naviglio (district of Milan),0.010638,2,45.44763,9.154580
77,Villapizzone,0.010000,2,45.49834,9.144530
